In [1]:
import os
import json
from lib import StyloAnalyzer

corpora = ['aurthur_conan_doyle', 'charles_dickens',
           'h_g_wells', 'jane_austen', 'jonathan_swift']
# corpora = ['aurthur_conan_doyle']


In [2]:
class StyloAnalyzer(StyloAnalyzer):
    def __init__(self, label: str, text: str) -> None:
        super().__init__(label, text)

In [3]:
analyzed_data = {}
for corpus in corpora:
    for entry in os.scandir(os.path.join('corpora_split', corpus)):
        if entry.is_file():
            with open(entry.path, 'r') as file:
                analyzed_data[corpus + '.' + entry.name] = StyloAnalyzer(corpus + '.' + entry.name, file.read())
        # break


In [4]:
result = {}
for data in analyzed_data.values():
    result[data.label] = data.vector

In [5]:
json_object = json.dumps(result)

with open(os.path.join('output', 'metrics.json'), 'w') as outfile:
    outfile.write(json_object)

In [6]:
result = {}
for data in analyzed_data.values():
    result[data.label] = data.json

json_object = json.dumps(result)
with open(os.path.join('output', 'plot.json'), 'w') as outfile:
    outfile.write(json_object)